In [1]:
import pandas as pd
import random
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#import pydot
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import gc
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D,Conv2D, Conv1D, AveragePooling2D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import sys
import boto3
from boto3.s3.transfer import TransferConfig
import tensorflow as tf
import tensorflow as tf

## 1. Import Embeddings (X) and Labels (Y)

In [3]:
now = time.time()
infile = None
infile = open("embeddings.p",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

342.367484331131


22

In [4]:
embeddings_np_complete = np.array(list(embeddings.items()))
del(embeddings)

## 2. Load chapter model and helper functions

In [5]:
#Load chapeter model weights
chapter_model = None
json_file = open('model_num_cnn2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
chapter_model = model_from_json(loaded_model_json)
opt = keras.optimizers.Adam(lr=0.001)
# load weights into new model
chapter_model.load_weights("model_num_cnn2.h5")
chapter_model.trainable = False

In [13]:
def padding(embeddings):
    max_sentence = 20
    padding = max_sentence - len(embeddings)
    if padding > 0:
        padding_shape = (padding, 256, 768)
        pad = np.zeros(padding*256*768).reshape(padding_shape)
        return np.append(embeddings, pad, axis = 0).astype('float16')
    else:
        return embeddings
    
def split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat):
    HADM_ID_DF = None
    loaded_model = None
    HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])
    CHAPTER_PIVOT_DF_COPY = HADM_ID_DF.set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='left').copy(deep=False)
    CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF_COPY)
    return train_test_split(embeddings_concat, CHAPTER_PIVOT_NP, test_size=0.3)

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    #p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    #r = tn / (tn + fn + K.epsilon())
    acc = (tp + tn)/(tp+tn+fp+fn+K.epsilon())
    #f1 = 2*p*r / (p+r+K.epsilon())
    f1 = 2*p*r / (p+r+K.epsilon())
    f2 = 2*f1*acc/(f1+acc+K.epsilon())
    #f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    f2 = tf.where(tf.math.is_nan(f2), tf.zeros_like(f2), f2)

    return 1 - K.mean(f2)

def get_model(label_len):
    #define model - STATIC LAYERS
    model = Sequential()
    for layer in chapter_model.layers[:4]:
        layer.trainable = False
        model.add(layer)
    #model.add(Conv1D(label_len, 8))
    #model.add(AveragePooling1D(pool_size=4))
    #model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2048, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(label_len, activation = 'sigmoid'))

    model = Model(inputs=model.input, outputs=model.output)
    return model

## 3. Loop on individual therapy area (bag of models)

In [14]:
score_list = []
history_list = []
file_name = 'adverse_effect.csv'

CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
print(file_name+' orginal shape: '+str(embeddings_np.shape))
HADM_ID_LIST = embeddings_np[:,0].copy()
embeddings = embeddings_np[:,1].copy()
    
#Zero Pad embeddings
embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
del(embeddings)
gc.collect()
print(file_name+' Padding complete')

#Reshape embeddings
#embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
print(file_name+' modified shape: '+str(embeddings_concat.shape))
del(embeddings_padded)
gc.collect()
    
X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
X_val = X_test2[:int(0.25*len(X_test2))]
y_val = y_test2[:int(0.25*len(y_test2))]
del(HADM_ID_LIST)
del(CHAPTER_PIVOT_DF)
del(embeddings_concat)
gc.collect()

adverse_effect.csv orginal shape: (6339, 2)
adverse_effect.csv Padding complete
adverse_effect.csv modified shape: (6339, 5120, 768)
Train and Test Shapre: (4437, 5120, 768), (1902, 5120, 768), (4437, 5), (1902, 5)


0

In [17]:
#call model
label_len = y_train2.shape[1]
model = get_model(label_len)
opt = keras.optimizers.Adam(lr=0.00001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
#f1_loss
#model.compile(loss=f1_loss, optimizer=opt, metrics=[get_f1, 'accuracy'])
flat_X_train = model.fit(X_train2, y_train2, epochs=5, verbose = 1)
#, y_train2, epochs=4,batch_size=32, verbose=1, validation_data = (X_val,y_val))

Epoch 1/5
4437/4437 [==============================] - 185s 42ms/step - loss: 0.6656 - get_f1: 0.2685 - accuracy: 0.6501
Epoch 2/5
4437/4437 [==============================] - 186s 42ms/step - loss: 0.6236 - get_f1: 0.2443 - accuracy: 0.6719
Epoch 3/5
4437/4437 [==============================] - 186s 42ms/step - loss: 0.6063 - get_f1: 0.2533 - accuracy: 0.6843
Epoch 4/5
4437/4437 [==============================] - 187s 42ms/step - loss: 0.5944 - get_f1: 0.2697 - accuracy: 0.6907
Epoch 5/5
4437/4437 [==============================] - 186s 42ms/step - loss: 0.5787 - get_f1: 0.2841 - accuracy: 0.6989


In [16]:
model.evaluate(X_test2,y_test2, verbose = 1)

1902/1902 [==============================] - 61s 32ms/step


[0.5602686459083036, 0.43674951791763306, 0.7054677605628967]

In [12]:
model_json = model.to_json()
with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")

In [6]:
pickle.dump( embeddings, open( "/home/ec2-user/SageMaker/embeddings.p", "wb" ) )

## 4. Random Forest

In [70]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [71]:
rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf = 5)

In [72]:
rf_model.fit(flat_X_train, y_train2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [73]:
rf_model.score(flat_X_train, y_train2)

0.5164661802405837

In [74]:
rf_model.score(flat_X_test, y_test2)

0.14037854889589904

In [ ]:
gb_model = GradientBoostingClassifier()
gb_model.fit(flat_X_train, y_train2[:,0])

In [ ]:
gb_model.score(flat_X_train, y_train2[,:0])

In [ ]:
gb_model.score(flat_X_test, y_test2[:,0])

In [61]:
y_test2[10:20]

array([[0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])